While the analysis of the Beatles song 'yesterday' gave back results which are in line with our hypothesis (2.1-beta2 is better, flac is better), it is still dependent on the judgement that the song is sad (while the lyrics are sad, the acoustic qualities might not be). Thus, it can be beneficial to find another way of determining this accuracy percentage for different codecs and versions.

To avoid having to specify a 'ground truth' label, we can also look at the binary agreement between mood_sad and mood_happy. Since the combination of these two *roughly* approaches valence (both negative is a valence of 0), we can use the label probabilities from these two classifiers to check if the output is consistent and correct. i.e. we do the following:

## Valid combinations (will be considered 'correct' for any submission)
- (mood_happy, happy) + (mood_sad, not_sad) = positive valence
- (mood_happy, not_happy) + (mood_sad, sad) = negative valence
- (mood_happy, not_happy) + (mood_sad, not_sad) = neutral valence

## Invalid combinations (will be considered 'incorrect' for any submission
- (mood_happy, happy) + (mood_sad, sad) = ?

In [36]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import numpy as np
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(15, 15)})

from tqdm.notebook import tqdm
tqdm().pandas()

# Load in the acousticbrainz dataset into the variable 'acousticbrainz' (V3 also include build SHA to make sure that software
# version is the same on a commit basis.
acousticbrainz = pd.read_hdf(Path.cwd() / 'datasets' / 'acousticbrainzV3.h5')

def get_accuracy(df):
    incorrect = (df[('mood_happy', 'happy')] > 0.5) & (df[('mood_sad', 'sad')] > 0.5)
    count = incorrect.value_counts()[False]
    return (count, len(incorrect), count/len(incorrect))

def print_accuracy(groupname, accuracy):
    print(f"{groupname}: {full_dataset[0]}/{full_dataset[1]} = {full_dataset[2] * 100}%")
    

In [83]:
accuracies = pd.DataFrame(columns=['Correct', 'Total', 'Percentage'])

# Run for every group > 10.000
grouped = acousticbrainz.groupby(['bit_rate', 'codec', 'essentia_low', 'essentia_git_sha_low', 'essentia_build_sha_low'])
tocheck = grouped.size()[(grouped.size() > 10000)]
display(tocheck)

for groupname, group in grouped:
    if groupname in tocheck:
        acc = get_accuracy(group)
        accuracies = accuracies.append(pd.Series([acc[0], acc[1], acc[2]], index=accuracies.columns, name=groupname))

accuracies.sort_values(by='Percentage', ascending=False)

bit_rate  codec   essentia_low  essentia_git_sha_low          essentia_build_sha_low                  
0         flac    2.1-beta1     v2.1_beta1-28-g21ef5f4        5c9e65ec81695c916afb4cba055c226bfc528ee6     36800
                                v2.1_beta1-28-g21ef5f4-dirty  ca57ba49d9b1854bd80e60cf9ccf267278fb7d6b     90453
                                v2.1_beta1-6-g5578087         5a55991247aa8bc5301991e5ebe36927b56abb96     17292
                                                              7598a69e7b456135ce4c070efb04a3d0ebb71fdd     10171
                                                              7f154e5731828a63663f0144680ef47bd640f88b     28706
                                v2.1_beta1-7-ge0e83e8-dirty   12879b851c9f4b0d899e1af4e221aa110f283a84     13391
                                                              50a0fbec89d6a9cedea3d45b6611406f7e8c7b1a     14636
                  2.1-beta2     v2.1_beta2                    70f2e5ece6736b2c40cc944ad0e695b16b925413    

,Correct,Total,Percentage
"(128000, mp3, 2.1-beta1, v2.1_beta1-28-g21ef5f4-dirty, ca57ba49d9b1854bd80e60cf9ccf267278fb7d6b)",31589.0,31840.0,0.992117
"(320000, mp3, 2.1-beta1, v2.1_beta1-28-g21ef5f4-dirty, ca57ba49d9b1854bd80e60cf9ccf267278fb7d6b)",31297.0,31570.0,0.991353
"(192000, mp3, 2.1-beta1, v2.1_beta1-28-g21ef5f4-dirty, ca57ba49d9b1854bd80e60cf9ccf267278fb7d6b)",35447.0,35953.0,0.985926
"(192000, mp3, 2.1-beta1, v2.1_beta1-34-gf95df8b, 5a811adb4c06fd6d02ca903f7349a26d851abfa8)",10418.0,10582.0,0.984502
"(256000, vorbis, 2.1-beta2, v2.1_beta2, 70f2e5ece6736b2c40cc944ad0e695b16b925413)",13236.0,13466.0,0.982920
"(160000, mp3, 2.1-beta1, v2.1_beta1-28-g21ef5f4-dirty, ca57ba49d9b1854bd80e60cf9ccf267278fb7d6b)",11574.0,11788.0,0.981846
"(192000, mp3, 2.1-beta2, v2.1_beta2, 70f2e5ece6736b2c40cc944ad0e695b16b925413)",23044.0,24057.0,0.957892
"(320000, mp3, 2.1-beta2, v2.1_beta2, 70f2e5ece6736b2c40cc944ad0e695b16b925413)",10918.0,11516.0,0.948072
"(128000, mp3, 2.1-beta2, v2.1_beta2, 70f2e5ece6736b2c40cc944ad0e695b16b925413)",15990.0,16903.0,0.945986
"(0, flac, 2.1-beta1, v2.1_beta1-28-g21ef5f4-dirty, ca57ba49d9b1854bd80e60cf9ccf267278fb7d6b)",85345.0,90453.0,0.943529
